In [2]:
import gym
from gym import Env
from gym.spaces import Box, Dict, Tuple, MultiBinary, MultiDiscrete, Discrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv 
from stable_baselines3.common.evaluation import evaluate_policy

In [15]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60

    def step(self, action):
        self.state += action-1
        self.shower_length -= 1

        if self.state >=37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <=0:
            done = True
        else:
            done = False
        
        info = {}
        return self.state, reward, done, info
    def render(self):
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [16]:
env = ShowerEnv()

In [140]:
env.action_space.sample()

2

In [143]:
model = PPO('MlpPolicy', env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [148]:
model.learn(total_timesteps=60000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -27      |
| time/              |          |
|    fps             | 526      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -15.7         |
| time/                   |               |
|    fps                  | 364           |
|    iterations           | 2             |
|    time_elapsed         | 11            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00040461836 |
|    clip_fraction        | 0.00264       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.242        |
|    explained_variance   | 0.0407        |


In [164]:
evaluate_policy(model, env, n_eval_episodes=10)

(12.0, 58.787753826796276)

In [222]:
episode = 5
for n in range(1,episode+1):
    obs = env.reset()
    score = 0
    done = False
    while not done:
        action = model.predict(obs)
        obs, reward, done, info = env.step(action[0])
        score += reward
    print(f"episode - {n} || score - {score}")

episode - 1 || score - 46
episode - 2 || score - 60
episode - 3 || score - -60
episode - 4 || score - 60
episode - 5 || score - 40
